In [ ]:
"""
Animal Shelter Dashboard (Dash + MongoDB)

This module implements an interactive web dashboard for exploring
animal shelter outcome data stored in MongoDB. The dashboard supports:

- Environment-based credential loading
- Server-side pagination
- Projection-based queries
- Filtered rescue-category views
- Server-side aggregation for analytics
- Interactive table, pie chart, and geolocation map

Author: Eric Foster
"""

# ============================
# Standard Library Imports
# ============================

import math
import os
import base64

# ============================
# Third-Party Imports
# ============================

from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, callback_context
from dash import dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
from dotenv import load_dotenv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ============================
# Local Application Imports
# ============================

# CRUD abstraction layer for MongoDB
from DB_CRUD_Python_Module import Animal_Shelter


# ============================================================
# Environment Configuration
# ============================================================

# Load variables from .env file into environment
load_dotenv()

# Database credentials pulled from environment variables
username = os.environ.get("MONGO_USER")
password = os.environ.get("MONGO_PASS")
host = os.environ.get("MONGO_HOST", "127.0.0.1")
port = int(os.environ.get("MONGO_PORT", "27017"))
database = os.environ.get("MONGO_DB", "aac")
collection = os.environ.get("MONGO_COL", "animals")
auth_source = os.environ.get("MONGO_AUTHSOURCE", database)

# Initialize CRUD interface
db = Animal_Shelter(username, password, host, port, database, collection, auth_source=auth_source)


# ============================================================
# MongoDB Projection Definition
# ============================================================

# Limits returned fields for performance and UI stability
projection = {
    "_id": 0,
    "animal_type": 1,
    "breed": 1,
    "sex_upon_outcome": 1,
    "age_upon_outcome_in_weeks": 1,
    "name": 1,
    "location_lat": 1,
    "location_long": 1
}

# Initial fetch to build table columns
records = db.read({}, projection=projection, limit=100)
df = pd.DataFrame.from_records(records)


# ============================================================
# Predefined Rescue Filter Queries
# ============================================================

FILTER_QUERIES = {
    "Water Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Mountain Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}
}


# ============================================================
# Dash Application Initialization
# ============================================================

app = JupyterDash(__name__)

# Load organization logo and encode as base64
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# ============================================================
# Dashboard Layout
# ============================================================

app.layout = html.Div([

    # Invisible div used for future extensions
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Header Section
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Animal Shelter Dashboard'))),
    html.Center(html.H4('Created by: Eric Foster')),
    html.Hr(),

    # Rescue Filter Selector
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'],
            value='Reset',
            inline=True
        )
    ]),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_action="custom",
        page_current=0,
        page_size=10,
        page_count=0,
        filter_action="none",
        sort_action="none",
        row_selectable="single",
        selected_rows=[0],
    ),

    html.Br(),
    html.Hr(),

    # Graph + Map Row
    html.Div(
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', style={'width': '50%'}),
            html.Div(id='map-id', style={'width': '50%'})
        ]
    )
])


# ============================================================
# Server-Side Pagination Callback
# ============================================================

@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "page_count"),
    Output("datatable-id", "page_current"),
    Input("filter-type", "value"),
    Input("datatable-id", "page_current"),
    Input("datatable-id", "page_size"),
)
def update_table(filter_type, page_current, page_size):
    """
    Retrieves only the records required for the current page.
    Prevents loading the entire dataset into memory.
    """

    query = FILTER_QUERIES.get(filter_type, {})

    # Reset to first page when filter changes
    triggered = callback_context.triggered[0]["prop_id"].split(".")[0]
    if triggered == "filter-type":
        page_current = 0

    total_rows = db.count(query)
    page_count = max(1, math.ceil(total_rows / page_size))

    # Clamp page index
    if page_current >= page_count:
        page_current = page_count - 1

    skip = page_current * page_size

    records = db.read(
        query,
        projection=projection,
        limit=page_size,
        skip=skip
    )

    return records, page_count, page_current


# ============================================================
# Breed Distribution Pie Chart
# ============================================================

@app.callback(
    Output('graph-id', "children"),
    Input('filter-type', "value")
)
def update_graphs(filter_type):
    """
    Uses MongoDB aggregation to compute breed counts
    rather than computing client-side.
    """

    query = FILTER_QUERIES.get(filter_type, {})
    counts = db.breed_counts(query, limit=20)

    if not counts:
        return [html.P("No breed data available.")]

    dff = pd.DataFrame(counts)

    fig = px.pie(
        dff,
        names="breed",
        values="count",
        title=f"Preferred Breeds for Selected Rescue Type: {filter_type}"
    )

    return [dcc.Graph(figure=fig)]


# ============================================================
# Column Highlighting
# ============================================================

@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns"),
)
def update_styles(selected_columns):
    """
    Highlights selected column for better UX.
    """

    selected_columns = selected_columns or []

    return [
        {"if": {"column_id": col}, "background_color": "#D2F3FF"}
        for col in selected_columns
    ]


# ============================================================
# Geolocation Map Callback
# ============================================================

@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")
)
def update_map(viewData, selected_rows):
    """
    Displays map marker for selected animal.
    """

    if not viewData:
        return [html.P("No data available to map.")]

    dff = pd.DataFrame.from_records(viewData)

    row = selected_rows[0] if selected_rows else 0

    required = {"location_lat", "location_long", "breed", "name"}
    if not required.issubset(dff.columns):
        return [html.P("Missing required location fields.")]

    lat = dff.loc[row, "location_lat"]
    lon = dff.loc[row, "location_long"]

    if pd.isna(lat) or pd.isna(lon):
        return [html.P("No GPS coordinates available.")]

    breed = dff.loc[row, "breed"]
    name = dff.loc[row, "name"]

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[lat, lon],
            zoom=10,
            children=[
                dl.TileLayer(),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(str(breed)),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(name))
                        ])
                    ]
                )
            ]
        )
    ]


# ============================================================
# Application Entry Point
# ============================================================

app.run_server()


Dash app running on http://127.0.0.1:8050/
